# Beyond Basic Cleaning: Preparando Dados para Machine Learning

Noteboo que terá alterações pessoais para acompanhar o original acesse os links dos readme.


## 1) CLoad the dataset
We'll start with a deliberately **messy** dataset that contains missing values, outliers, and inconsistent date formats.

## 1) Carregar o dataset
Começaremos com um dataset propositalmente **bagunçado**, que contém valores ausentes, outliers e formatos de data inconsistentes.


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('ml_prep_demo_raw.csv')
df.head()

,id,event_date,age,income,gender,region,purchases,category,churn
0,1,2023-04-13,34.0,NaN,Female,East,2.0,D,0
1,2,2023-12-15,40.0,54770.0,Male,East,2.0,NaN,1
2,3,2023-09-28,NaN,85399.0,Female,East,2.0,B,0
3,4,2023-04-17,44.0,52703.0,Male,NaN,NaN,B,1
4,5,2023-03-13,NaN,53504.0,NaN,North,7.0,B,0


```py
import warnings

 warnings.filterwarnings("ignore")
```

A importação e código acima faz com que todos os warnings do Python sejam suprimidos durante a execução do código.

Mesmo sendo um material didático acho sensato o aluno(a) já ver os erros, para no momento não traver ou se assustar e não saber o que fazer.
claro que é a minha opnião pessoal.


## 2) Quick EDA & data types
Check shape, nulls, and dtypes to see what we’re dealing with.

## 2) EDA rápido e tipos de dados
Vamos verificar o shape, valores nulos e dtypes para entender com o que estamos lidando.


In [2]:
print(df.shape) # mostra de forma resumida linha, coluna
df.info() # informações básicas do dataset
df.isna().mean().sort_values(ascending=False)  # média de valores nulos por coluna

(1500, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1500 non-null   int64  
 1   event_date  1500 non-null   object 
 2   age         1405 non-null   float64
 3   income      1396 non-null   float64
 4   gender      1379 non-null   object 
 5   region      1414 non-null   object 
 6   purchases   1397 non-null   float64
 7   category    1396 non-null   object 
 8   churn       1500 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 105.6+ KB


gender        0.080667
income        0.069333
category      0.069333
purchases     0.068667
age           0.063333
region        0.057333
id            0.000000
event_date    0.000000
churn         0.000000
dtype: float64

## 3) Fix dates & basic schema
The `event_date` has mixed formats. We'll coerce to datetime safely; invalid ones become `NaT`.

## 3) Corrigindo datas e esquema básico
A coluna `event_date` possui formatos mistos. Vamos converter para datetime de forma segura; valores inválidos se tornarão `NaT` (Not a Time).


In [3]:
df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')# tenta converter cada valor para datetime; se não conseguir, coloca NaT.
df[['event_date']].head(10)# mostra as primeiras linhas da coluna convertida para checar se a conversão funcionou.

,event_date
0,2023-04-13
1,2023-12-15
2,2023-09-28
3,2023-04-17
4,2023-03-13
5,2023-07-08
6,2023-01-21
7,2023-04-13
8,2023-05-02
9,2023-08-03


## 4) Feature engineering from dates
Create features like `event_month`, `event_dayofweek`, and `recency_days` (relative to dataset max date).

## 4) Criação de features a partir de datas
Vamos criar novas features, como `event_month`, `event_dayofweek` e `recency_days` (relativa à data máxima do dataset)features


In [4]:
max_date = df['event_date'].max()  # encontra a data máxima do dataset
df['event_month'] = df['event_date'].dt.month  # mês do evento
df['event_day_of_week'] = df['event_date'].dt.dayofweek  # dia da semana do evento (0=segunda, 6=domingo)
df['recency_days'] = (max_date - df['event_date']).dt.days  # diferença em dias em relação à data máxima

df[['event_date','event_month','event_day_of_week','recency_days']].head()  # mostra as primeiras linhas das novas features

,event_date,event_month,event_day_of_week,recency_days
0,2023-04-13,4.0,3.0,297.0
1,2023-12-15,12.0,4.0,51.0
2,2023-09-28,9.0,3.0,129.0
3,2023-04-17,4.0,0.0,293.0
4,2023-03-13,3.0,0.0,328.0


## 5) Split features & target
We'll predict `churn` as the target. Identify numeric vs categorical columns.

## 5) Separando features e target
Nosso objetivo srr á **prever `churn`** como variável alvo (target).  
Vamos também identificar **colunas numéricas** e **colunas categóricas*

 diretamente**.
*.

In [5]:
target = 'churn'
feature_cols = [c for c in df.columns if c not in [target]]  # todas as colunas, exceto a target
X = df[feature_cols].copy()  # features
y = df[target].copy()        # target

# Colunas numéricas
numeric_features = ['age','income','purchases','event_month','event_day_of_week','recency_days']

# Colunas categóricas
categorical_features = ['gender','region','category']

# Colunas de data (serão descartadas após extrair features)
date_cols = ['event_date']  

# Remove a coluna de data bruta das features
X = X.drop(columns=date_cols)

X.head()  # mostra as primeiras linhas das features

,id,age,income,gender,region,purchases,category,event_month,event_day_of_week,recency_days
0,1,34.0,NaN,Female,East,2.0,D,4.0,3.0,297.0
1,2,40.0,54770.0,Male,East,2.0,NaN,12.0,4.0,51.0
2,3,NaN,85399.0,Female,East,2.0,B,9.0,3.0,129.0
3,4,44.0,52703.0,Male,NaN,NaN,B,4.0,0.0,293.0
4,5,NaN,53504.0,NaN,North,7.0,B,3.0,0.0,328.0


### 🎯 Target

- `churn` será a variável que queremos prever.  
- `y` recebe apenas a coluna alvo.

### 🧩 Features

- `X` contém todas as colunas que serão usadas como entrada para o modelo.  
- Aqui separamos manualmente **numéricas**, **categóricas** e **datas**.

### 🗓️ Por que dropar `event_date`

- Já extraímos informações relevantes da data:  
  - `event_month`  
  - `event_day_of_week`  
  - `recency_days`  
- A coluna bruta `event_date` **não é útil para ML diretamente**.


## 6) Outlier exploration (numeric)
Use Z-score/IQR to **detect** outliers before deciding to cap/winsorize.

## 6) Exploração de outliers (colunas numéricas)

Antes de decidir **como tratar outliers** (por exemplo, cap ou winsorize), vamos **identificá-los** usando Z-score/IQR.


In [6]:
from scipy import stats
import numpy as np

# Seleciona apenas as features numéricas
numX = X[numeric_features].copy()

# Calcula o Z-score absoluto, ignorando NaNs
z = np.abs(stats.zscore(numX, nan_policy='omit'))

# Máscara para detectar linhas com pelo menos um outlier (Z-score > 3)
outlier_mask = (z > 3).any(axis=1)

# Taxa de outliers no dataset
outlier_rate = outlier_mask.mean()
outlier_rate

0.012

Explicação didática

Z-score: mede quantos desvios-padrão cada valor está distante da média.

Valores com |Z| > 3 são geralmente considerados outliers extremos.

nan_policy='omit': ignora valores ausentes na hora de calcular o Z-score, evitando erros.

outlier_mask: identifica quais linhas têm pelo menos um outlier.

outlier_rate: mostra a proporção de linhas com outliers, ajudando a decidir se vamos tratar, remover ou deixar os outliers.

## 7) Preprocessing pipeline
- **Impute** missing values (KNN for numeric; constant for categorical)
- **Scale** numeric features
- **One-hot encode** categoricals

We’ll wrap everything in a scikit-learn **Pipeline** to avoid leakage and ensure repeatability.

a tradução ficará abaixo do bloco do código, pois irá integrar uma explicação.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Split inicial (com estratificação para manter proporção de churn)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Transformações para colunas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3)),   # preenche valores ausentes usando vizinhos
    ('scaler', StandardScaler())              # normaliza escala
])

# Transformações para colunas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # preenche categorias ausentes
    ('encoder', OneHotEncoder(handle_unknown='ignore'))    # one-hot encoding
])

# Junta tudo em um pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


## 7) Pipeline de Pré-processamento

- Imputação (preencher valores ausentes):
- - Numéricos → KNNImputer (usa vizinhos próximos para preencher).
  - Categóricos → SimpleImputer(strategy='most_frequent') (preenche com a categoria mais frequente).
- Escalonamento (numeric):
- - StandardScaler() → deixa cada feature com média 0 e desvio padrão 1.
- Codificação one-hot (categorical):
- - OneHotEncoder(handle_unknown='ignore') → transforma categorias em colunas binárias (0/1).
  - O parâmetro handle_unknown='ignore' evita erro se aparecer uma nova categoria no teste que não existia no treino.
- Tudo isso é empacotado em um Pipeline do scikit-learn, garantindo:
- - ✅ Reprodutibilidade
  - ✅ Nada do teste vaza para o treino
  - ✅ Um único objeto controla todas as etapas

### Optional: Outlier capping (winsorization)
We can add a custom transformer to cap extreme values after imputation.

In [8]:
# Importa classes base para criar transformadores compatíveis com scikit-learn
from sklearn.base import BaseEstimator, TransformerMixin

# Criação de um transformador chamado Winsorizer
class Winsorizer(BaseEstimator, TransformerMixin):
    # Inicializa o transformador definindo os limites inferior e superior (quantis)
    def __init__(self, quantile_low=0.01, quantile_high=0.99):
        self.quantile_low = quantile_low       # ex: 1º percentil
        self.quantile_high = quantile_high     # ex: 99º percentil
        self.lows_ = None                      # limites inferiores calculados
        self.highs_ = None                     # limites superiores calculados

    # Método de ajuste (fit): calcula os quantis com base nos dados
    def fit(self, X, y=None):
        import pandas as pd
        X = pd.DataFrame(X)                               # garante que X seja DataFrame
        self.lows_ = X.quantile(self.quantile_low)        # calcula o limite inferior
        self.highs_ = X.quantile(self.quantile_high)      # calcula o limite superior
        return self                                       # retorna o próprio objeto (padrão em sklearn)

    # Método de transformação (transform): aplica a Winsorização
    def transform(self, X):
        import pandas as pd
        X = pd.DataFrame(X)                               # garante que X seja DataFrame
        # Trava valores abaixo do low e acima do high em cada coluna
        X = X.clip(lower=self.lows_, upper=self.highs_, axis=1)
        return X.values                                   # retorna como numpy array (compatível com sklearn)


# Extra - Devo sempre usar o Winsorizer?
- Ao pesquisar se há diferença entre fazer na mão e usando o código winsorizer, não há quase diferenças e irei citar abaixo.

## Diferença principal 🚨
- Manual (clip): rápido, ótimo para análise exploratória e testes.
- Classe (Winsorizer): segue o padrão do scikit-learn → encaixa em Pipeline, pode ser usado dentro de GridSearchCV, etc.

Então depende do caso, como regras de negócio ou do problema abordado


## 8) Train a model with the preprocessing pipeline
We’ll use Logistic Regression as a baseline to demonstrate how preprocessing and modeling fit together.

## 8) Treine um modelo com o pipeline de pré-processamento
Usaremos a Regressão Logística como linha de base para demonstrar como o pré-processamento e a modelagem se encaixam.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

clf = Pipeline(steps=[('preprocess', preprocessor), ('model', LogisticRegression(max_iter=200))])
# Cria um Pipeline com duas etapas:
# 'preprocess': aplica o preprocessor que você definiu antes (ex.: normalização, one-hot encoding, imputação).
# 'model': roda a regressão logística.
# O max_iter=200 aumenta o número de iterações para garantir que o modelo convirja.

clf.fit(X_train, y_train)
# Treina o pipeline inteiro:
# O preprocessor aprende no X_train (ex.: média para imputação, colunas para OneHotEncoder).
# A regressão logística é treinada nos dados transformados.

preds = clf.predict(X_test) # preds: rótulos 0 ou 1 previstos.
probs = clf.predict_proba(X_test)[:,1] # probs: probabilidades da classe positiva ([:,1] pega só a coluna referente à classe 1).

acc = accuracy_score(y_test, preds) # acc: porcentagem de acertos.
auc = roc_auc_score(y_test, probs) # auc: métrica que avalia a capacidade do modelo em separar classes (independente de threshold fixo).

print("Accuracy:", round(acc, 4))
print("ROC AUC:", round(auc, 4))
print("\nClassification Report:\n", classification_report(y_test, preds))

Accuracy: 0.6833
ROC AUC: 0.5135

Classification Report:
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       205
           1       0.00      0.00      0.00        95

    accuracy                           0.68       300
   macro avg       0.34      0.50      0.41       300
weighted avg       0.47      0.68      0.55       300



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


⚠️ **UndefinedMetricWarning**  
Esse aviso aparece quando uma **classe do dataset não foi prevista pelo modelo**.  
Exemplo: se a classe `1` existe nos dados, mas o modelo só previu `0`, a precisão para `1` não pode ser calculada → é definida como `0.0`.  

👉 Isso geralmente indica **desbalanceamento de classes** ou que o modelo não está aprendendo a distinguir bem essa classe.  


### Compare with winsorization variant

In [10]:
# Criação de um Pipeline que combina pré-processamento e modelo
clf_w = Pipeline(steps=[
    ('preprocess', preprocessor),            # etapa de pré-processamento (ex: normalização, one-hot encoding, etc.)
    ('model', LogisticRegression(max_iter=200))  # modelo de Regressão Logística com até 200 iterações para convergir
])

# Treinamento do modelo com os dados de treino
clf_w.fit(X_train, y_train)

# Predição nos dados de teste:
# - preds_w: classes previstas (0 ou 1)
# - probs_w: probabilidades previstas para a classe positiva (coluna índice 1)
preds_w = clf_w.predict(X_test)
probs_w = clf_w.predict_proba(X_test)[:, 1]

# Cálculo das métricas:
# - Accuracy: porcentagem de acertos
# - ROC AUC: mede quão bem o modelo separa as classes (robusta em cenários desbalanceados)
acc_w = accuracy_score(y_test, preds_w)
auc_w = roc_auc_score(y_test, probs_w)

# Impressão dos resultados já arredondados para 4 casas decimais
print("Winsorized Accuracy:", round(acc_w, 4))
print("Winsorized ROC AUC:", round(auc_w, 4))


Winsorized Accuracy: 0.6833
Winsorized ROC AUC: 0.5135


###  ⁇  Why use `class_weight='balanced'`?
The parameter `class_weight='balanced'` in Logistic Regression automatically adjusts class weights **based on their frequency in the dataset**.  
- If class `1` is rare, it gets a higher weight during training.  
- This forces the model to pay more attention to the minority class, reducing the risk of predicting only the majority class.  

⚠️ Note: this doesn’t fully solve imbalance, but it **partially improves the situation**. The model treats classes more fairly, although overall *accuracy* might slightly drop.  
Often, additional techniques such as **oversampling, undersampling, or decision threshold tuning** are still required.


##⚖️ Por que usar `class_weight='balanced'`?ês
O parâmetro `class_weight='balanced'` na Regressão Logística ajusta automaticamente os pesos das classes **de acordo com sua frequência no dataset**.  
- Se a classe `1` é rara, ela recebe mais peso no treinamento.  
- Isso força o modelo a prestar mais atenção nela, reduzindo o risco de prever apenas a classe majoritária.  

⚠️ Importante: isso não resolve totalmente o desbalanceamento, mas **ajuda parcialmente**. O modelo passa a considerar as classes de forma mais justa, embora possa perder um pouco em *accuracy*.  
Em muitos casos, outras técnicas como **oversampling, undersampling ou ajuste de limiar de decisão** ainda são necessárias
---e still required.


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Definição do modelo de Regressão Logística dentro de um Pipeline
# - 'preprocess': aplica o pré-processamento definido anteriormente (ex: normalização, codificação, etc.)
# - 'model': cria a Regressão Logística
#   - max_iter=200: define o número máximo de iterações do otimizador (padrão é 100, aqui aumentamos para evitar não convergir)
#   - class_weight='balanced': ajusta os pesos automaticamente para lidar com desbalanceamento de classes
clf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=200, class_weight='balanced'))
])

# Treinamento do modelo com os dados de treino
clf.fit(X_train, y_train)

# Predições do modelo:
# - preds: predição final (classe 0 ou 1)
# - probs: probabilidade prevista para a classe positiva (classe 1)
preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)[:, 1]  # Pega apenas a coluna da classe positiva

# Avaliação do modelo:
# - Accuracy: proporção de acertos sobre todas as amostras
# - ROC AUC: mede a capacidade do modelo em distinguir entre classes (quanto mais perto de 1, melhor)
acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, probs)

# Impressão dos resultados:
# - Accuracy arredondado em 4 casas decimais
# - ROC AUC arredondado em 4 casas decimais
# - Classification Report: mostra precisão, recall, f1-score e suporte para cada classe
#   - zero_division=0 evita erros caso alguma métrica não possa ser calculada (ex: divisão por zero em recall)
print("Accuracy:", round(acc, 4))
print("ROC AUC:", round(auc, 4))
print("\nClassification Report:\n", classification_report(y_test, preds, zero_division=0))


Accuracy: 0.5367
ROC AUC: 0.5149

Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.56      0.62       205
           1       0.34      0.48      0.40        95

    accuracy                           0.54       300
   macro avg       0.52      0.52      0.51       300
weighted avg       0.59      0.54      0.55       300



## Antes (class_weight default)

- Classe 1 (churn):
- precision: 0.00
- recall: 0.00
- f1: 0.00

O modelo não previa nenhum churn.

## Depois (class_weight='balanced')

- Classe 1 (churn):
- precision: 0.34
- recall: 0.48
- f1: 0.40

Ou seja: ele erra bastante, mas finalmente começa a prever churn.

### 👉 O accuracy caiu de ~0.68 → ~0.54, mas isso é esperado:

Antes, ele só dizia "todo mundo é 0" e ganhava accuracy alto "de graça".

- Agora, ele tenta acertar os churns (classe difícil e minoritária).
e isso foi graças a LogisticRegression(max_iter=200, class_weight='balanced')
claro está longe do ideal mas agora o já vimos um progresso
e pelo que pesquisei da para usar  técnicas de oversampling/undersampling (SMOTE, etc. para tentarmos melhorar a qualidade da previsão..

## 9) Export the preprocessing pipeline
Save the fitted preprocessor + model for reuse.

In [12]:
import joblib
joblib.dump(clf, 'ml_prep_pipeline_baseline.joblib')
joblib.dump(clf_w, 'ml_prep_pipeline_winsor.joblib')
'Artifacts saved'

'Artifacts saved'